In [1]:
!pip install -qU langgraph langchain langchain-community langchain-ollama duckduckgo-search ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 116.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 6.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

!ollama pull llama3

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,228 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InReleas

In [3]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, END

# ----- Shared State -----
class AgentState(TypedDict):
    topic: str
    research_data: List[str]  # A list of findings
    blog_post: str            # The final output

In [4]:
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun

def researcher_node(state: AgentState):
    topic = state["topic"]
    print(f"Researcher is looking up: {topic}...")

    search = DuckDuckGoSearchRun()

    try:
        # You can tweak this query as you like
        results = search.run(f"key facts and latest news about {topic}")
    except Exception as e:
        results = f"Could not find data: {e}"

    print("Research complete.")

    # Only return the keys you want to update
    return {"research_data": state.get("research_data", []) + [results]}

In [5]:
# ChatOllama from langchain-ollama package
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

def writer_node(state: AgentState):
    print("Writer is drafting the post...")

    topic = state["topic"]
    data = state["research_data"][-1] if state["research_data"] else ""

    llm = ChatOllama(model="llama3", temperature=0.7)

    prompt = ChatPromptTemplate.from_template(
        """You are a tech blog writer.
Write a short, engaging blog post about "{topic}"
based ONLY on the following research data:

{data}

Return just the blog post content."""
    )

    chain = prompt | llm
    response = chain.invoke({"topic": topic, "data": data})

    print("Writing complete.")
    return {"blog_post": response.content}

In [6]:
# ----- Build the LangGraph -----
workflow = StateGraph(AgentState)

workflow.add_node("Researcher", researcher_node)
workflow.add_node("Writer", writer_node)

# Flow: Start -> Researcher -> Writer -> END
workflow.set_entry_point("Researcher")
workflow.add_edge("Researcher", "Writer")
workflow.add_edge("Writer", END)

app = workflow.compile()

In [7]:
if __name__ == "__main__":
    print("Starting the Multi-Agent System...\n")

    inputs: AgentState = {
        "topic": "The future of AI Agents",
        "research_data": [],
        "blog_post": "",
    }

    result = app.invoke(inputs)

    print("\n---------------- FINAL OUTPUT ----------------\n")
    print(result["blog_post"])

Starting the Multi-Agent System...

Researcher is looking up: The future of AI Agents...
Research complete.
Writer is drafting the post...
Writing complete.

---------------- FINAL OUTPUT ----------------

**The Future of AI Agents: 5 Trends That Will Reshape Business**

As we enter a new era of artificial intelligence (AI), one area that's gaining significant attention is AI agents. These intelligent assistants are poised to fundamentally change how businesses operate and create new value in the coming year. In this post, we'll dive into the top 5 trends shaping the future of AI agents, as outlined in Google Cloud's 2026 AI Agent Trends Report.

**Trend #1: Agents Take Action**

Big AI is no longer just about generating text or images; it's now focused on building and pushing toward the adoption of agents that can take actions and complete tasks on your behalf. This shift marks a significant tipping point in the AI revolution, as AI agents become an integral part of our daily lives.

